<a href="https://colab.research.google.com/github/PearlN04/ML-Personalised-Recommender-System/blob/main/RecommenderSystem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightfm

In [ ]:
!pip install matplotlib seaborn

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

file_path = '/content/drive/My Drive/PheliswaNontsanga_FNB_DataQuest_Challenge/data/clean_data/cleaned_data.csv'

df = pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,idcol,interaction,int_date,item,page,tod,item_type,item_descrip,segment,beh_segment,active_ind
0,755,DISPLAY,17-Jan-23,NONE,Screen1,Afternoon,ALL,DISPLAYED ALL ITEMS,segment3,B01,Semi Active
1,4521,DISPLAY,27-Feb-23,NONE,Screen1,Afternoon,ALL,DISPLAYED ALL ITEMS,segment1,B07,Semi Active
2,4521,DISPLAY,18-Feb-23,NONE,Screen1,Afternoon,ALL,DISPLAYED ALL ITEMS,segment1,B07,Semi Active
3,4521,DISPLAY,30-Jan-23,NONE,Screen1,Morning,ALL,DISPLAYED ALL ITEMS,segment1,B07,Semi Active
4,4521,CLICK,5-Feb-23,IBAB,Screen1,Afternoon,INSURE,GENERIC MESSAGE,segment1,B07,Semi Active


In [ ]:
!pip install lightfm
!pip install scipy
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd

file_path = '/content/drive/My Drive/PheliswaNontsanga_FNB_DataQuest_Challenge/data/clean_data/cleaned_data.csv'

df = pd.read_csv(file_path)
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,idcol,interaction,int_date,item,page,tod,item_type,item_descrip,segment,beh_segment,active_ind
0,755,DISPLAY,17-Jan-23,NONE,Screen1,Afternoon,ALL,DISPLAYED ALL ITEMS,segment3,B01,Semi Active
1,4521,DISPLAY,27-Feb-23,NONE,Screen1,Afternoon,ALL,DISPLAYED ALL ITEMS,segment1,B07,Semi Active
2,4521,DISPLAY,18-Feb-23,NONE,Screen1,Afternoon,ALL,DISPLAYED ALL ITEMS,segment1,B07,Semi Active
3,4521,DISPLAY,30-Jan-23,NONE,Screen1,Morning,ALL,DISPLAYED ALL ITEMS,segment1,B07,Semi Active
4,4521,CLICK,5-Feb-23,IBAB,Screen1,Afternoon,INSURE,GENERIC MESSAGE,segment1,B07,Semi Active


In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import coo_matrix, hstack
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# === 1. Map interaction types to numbers ===
interaction_map = {'DISPLAY': 0.1, 'CLICK': 1.0, 'CHECKOUT': 3.0}
df['interaction_score'] = df['interaction'].map(interaction_map)

# === 2. Encode user_id and item_id ===
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['idcol'])
df['item_id'] = item_encoder.fit_transform(df['item'])

# === 3. Prepare user metadata (drop duplicates) ===
user_features_df = df[['user_id', 'segment', 'beh_segment', 'active_ind', 'tod']].drop_duplicates(subset='user_id')
for col in ['segment', 'beh_segment', 'active_ind', 'tod']:
    user_features_df[col] = user_features_df[col].astype(str)

# === 4. Prepare item metadata (drop duplicates) ===
item_features_df = df[['item_id', 'item_type', 'item_descrip']].drop_duplicates(subset='item_id')
item_features_df['item_type'] = item_features_df['item_type'].astype(str)
item_features_df['item_descrip'] = item_features_df['item_descrip'].astype(str)

# === 5. Vectorize user features ===
user_dicts = user_features_df.drop('user_id', axis=1).to_dict(orient='records')
user_vec = DictVectorizer()
user_features = user_vec.fit_transform(user_dicts)

# === 6a. Vectorize item_type feature (categorical) ===
item_type_vec = DictVectorizer()
item_type_features = item_type_vec.fit_transform(
    item_features_df[['item_type']].to_dict(orient='records')
)

# === 6b. Vectorize item_descrip feature (textual) ===
tfidf_vec = TfidfVectorizer(max_features=300, stop_words='english')
item_desc_features = tfidf_vec.fit_transform(item_features_df['item_descrip'])

# === 6c. Combine item features ===
item_features = hstack([item_type_features, item_desc_features])

# === 7. Create interaction matrix ===
interactions = coo_matrix(
    (df['interaction_score'], (df['user_id'], df['item_id'])),
    shape=(user_features.shape[0], item_features.shape[0])
)

# === 8. Print shapes to verify ===
print("Interaction matrix shape:", interactions.shape)
print("User features shape:", user_features.shape)
print("Item features shape:", item_features.shape)

Interaction matrix shape: (84375, 104)
User features shape: (84375, 61)
Item features shape: (104, 170)


In [ ]:
from lightfm import LightFM
from lightfm.evaluation import precision_at_k, recall_at_k

# Create LightFM model with best-found hyperparameters
model = LightFM(loss='warp', no_components=100, learning_rate=0.05, user_alpha=1e-5, item_alpha=1e-5)

# Fit the model
model.fit(interactions,
          user_features=user_features,
          item_features=item_features,
          epochs=50,
          num_threads=4)

print("Model training complete!")

# Evaluate model
prec = precision_at_k(model, interactions,
                      user_features=user_features,
                      item_features=item_features,
                      k=10).mean()

rec = recall_at_k(model, interactions,
                  user_features=user_features,
                  item_features=item_features,
                  k=10).mean()

print(f"Precision@10: {prec:.4f}")
print(f"Recall@10:    {rec:.4f}")

Model training complete!
Precision@10: 0.1313
Recall@10:    0.4684


In [ ]:
#ndcg

import random
import numpy as np

def fast_ndcg_at_k(model, interactions, user_features=None, item_features=None, k=10, sample_size=1000):
    num_users, num_items = interactions.shape
    ndcg_scores = []

    sampled_users = random.sample(range(num_users), min(sample_size, num_users))

    for user_id in sampled_users:
        row = interactions.tocsr()[user_id]
        if row.nnz == 0:
            continue

        scores = model.predict(user_id, np.arange(num_items),
                               user_features=user_features,
                               item_features=item_features)

        top_k_items = np.argsort(-scores)[:k]
        actual = set(row.indices)

        dcg = sum([1.0 / np.log2(i + 2) for i, item in enumerate(top_k_items) if item in actual])
        ideal_dcg = sum([1.0 / np.log2(i + 2) for i in range(min(len(actual), k))])
        ndcg = dcg / ideal_dcg if ideal_dcg > 0 else 0.0

        ndcg_scores.append(ndcg)

    return np.mean(ndcg_scores)



In [ ]:
ndcg_sampled = fast_ndcg_at_k(model, interactions,
                              user_features=user_features,
                              item_features=item_features,
                              k=10, sample_size=1000)

print(f"NDCG@10 (sampled): {ndcg_sampled:.4f}")

NDCG@10 (sampled): 0.7738


In [ ]:
import numpy as np
from sklearn.metrics import pairwise_distances
from scipy.sparse import csr_matrix

def hit_rate_at_k(model, interactions, k=10, user_features=None, item_features=None):
    num_users, num_items = interactions.shape
    hits = 0
    for user_id in range(num_users):
        known_positives = interactions.tocsr()[user_id].indices
        if len(known_positives) == 0:
            continue
        scores = model.predict(user_id, np.arange(num_items), user_features=user_features, item_features=item_features)
        top_k = np.argsort(-scores)[:k]
        if np.intersect1d(top_k, known_positives).size > 0:
            hits += 1
    return hits / num_users

def item_diversity(item_features):
    similarity = 1 - pairwise_distances(item_features, metric="cosine")
    upper_triangle_indices = np.triu_indices_from(similarity, k=1)
    diversity = 1 - similarity[upper_triangle_indices].mean()
    return diversity

def novelty_score(model, interactions, k=10, user_features=None, item_features=None):
    num_users, num_items = interactions.shape
    item_popularity = np.array(interactions.sum(axis=0)).flatten()
    item_popularity += 1e-9  # avoid division by zero
    item_popularity = item_popularity / item_popularity.sum()
    novelty = 0.0
    for user_id in range(num_users):
        scores = model.predict(user_id, np.arange(num_items), user_features=user_features, item_features=item_features)
        top_k_items = np.argsort(-scores)[:k]
        novelty += -np.sum(np.log2(item_popularity[top_k_items]))
    return novelty / num_users

# Evaluate
print("Evaluating metrics...")
print("Hit Rate@10:", hit_rate_at_k(model, interactions, k=10, user_features=user_features, item_features=item_features))
print("Diversity:", item_diversity(item_features.toarray()))
print("Novelty:", novelty_score(model, interactions, k=10, user_features=user_features, item_features=item_features))

Evaluating metrics...
Hit Rate@10: 0.950802962962963
Diversity: 0.9092696448372373
Novelty: 52.805477
